# Deep Learning

In this notebook, I'll work with the churn dataset and use ANN and CNN to detect patterns.

I'll first get the dataset for CNN, extract it and then setup the GPU usage.

In [18]:
!wget https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P14-Convolutional-Neural-Networks.zip

--2019-03-30 04:26:12--  https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P14-Convolutional-Neural-Networks.zip
Resolving sds-platform-private.s3-us-east-2.amazonaws.com (sds-platform-private.s3-us-east-2.amazonaws.com)... 52.219.100.56
Connecting to sds-platform-private.s3-us-east-2.amazonaws.com (sds-platform-private.s3-us-east-2.amazonaws.com)|52.219.100.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232607759 (222M) [application/zip]
Saving to: ‘P14-Convolutional-Neural-Networks.zip.1’

P14-Convolutional-N   0%[                    ] 186.43K  54.3KB/s    eta 69m 44s^C


In [2]:
!unzip P14-Convolutional-Neural-Networks.zip

unzip:  cannot find or open P14-Convolutional-Neural-Networks.zip, P14-Convolutional-Neural-Networks.zip.zip or P14-Convolutional-Neural-Networks.zip.ZIP.


In [3]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

## Artificial Neural Networks

### Data Processing

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1]
y = dataset.iloc[:, -1]

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelEncoder_1 = LabelEncoder()
X.iloc[:, 1] = labelEncoder_1.fit_transform(X.iloc[:, 1])
labelEncoder_2 = LabelEncoder()
X.iloc[:, 2] = labelEncoder_2.fit_transform(X.iloc[:, 2])
oneHotEncoder = OneHotEncoder(categorical_features = [1])
X = oneHotEncoder.fit_transform(X).toarray()

# Avoid dummy variable trap
X = X[:, 1:]

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
X_train = standardScaler.fit_transform(X_train)
X_test = standardScaler.transform(X_test)

### Build the ANN

In [9]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras.backend as K
K.set_image_dim_ordering('tf')
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [10]:
ann_classifier = Sequential()

# Nodes count = average of input and output values

# First hidden layer
ann_classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = X_train.shape[1])) # relu is for rectifier
# Second hidden layer
ann_classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu')) # relu is for rectifier
# Output layer
ann_classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

ann_classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # adam is an optimized scholastic gradient descent

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.


In [11]:
ann_classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 1s 135us/step - loss: 0.4835 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 99us/step - loss: 0.4254 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 98us/step - loss: 0.4180 - acc: 0.8226
Epoch 4/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4127 - acc: 0.8292
Epoch 5/100
8000/8000 [==============================] - 1s 127us/step - loss: 0.4092 - acc: 0.8335
Epoch 6/100
8000/8000 [==============================] - 1s 110us/step - loss: 0.4065 - acc: 0.8349
Epoch 7/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.4042 - acc: 0.8339
Epoch 8/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4026 - acc: 0.8345
Epoch 9/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.4012 - acc: 0.8339
Epoch 10/100
8000/8000 [==============================] -

8000/8000 [==============================] - 1s 112us/step - loss: 0.3423 - acc: 0.8606
Epoch 81/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.3420 - acc: 0.8620
Epoch 82/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.3431 - acc: 0.8595
Epoch 83/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.3412 - acc: 0.8602
Epoch 84/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.3412 - acc: 0.8609
Epoch 85/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.3428 - acc: 0.8609
Epoch 86/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.3421 - acc: 0.8584
Epoch 87/100
8000/8000 [==============================] - 1s 94us/step - loss: 0.3417 - acc: 0.8604
Epoch 88/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.3413 - acc: 0.8595
Epoch 89/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.3413 - acc: 0.8595
Epoch 90/100

### Making predictions

In [12]:
y_pred = ann_classifier.predict(X_test)
y_pred = y_pred > 0.5

In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score

print("Accuracy: {}%".format(accuracy_score(y_test, y_pred)*100))
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix: ")
print("{}".format(cm))

Accuracy: 86.35000000000001%
Confusion matrix: 
[[1538   57]
 [ 216  189]]


## Convolutional Neural Network

In [14]:
from keras.layers import Convolution2D, MaxPooling2D, Flatten

### Build CNN

1. Convolution
2. Max Pooling
3. Flatten
4. Full Connection

In [15]:
classifier = Sequential()
classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
classifier.add(Flatten())
classifier.add(Dense(activation = 'relu', units=128))
classifier.add(Dense(activation = 'sigmoid', units=1))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Image Augmentation

In [16]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
                 rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_generator = ImageDataGenerator(rescale = 1./255)

training_set = train_generator.flow_from_directory('Convolutional_Neural_Networks/dataset/training_set', 
                                                    target_size = (64, 64), 
                                                    batch_size = 32, 
                                                    class_mode = 'binary')

test_set = train_generator.flow_from_directory('Convolutional_Neural_Networks/dataset/test_set', 
                                                target_size = (64, 64), 
                                                batch_size = 32, 
                                                class_mode = 'binary')

FileNotFoundError: [Errno 2] No such file or directory: 'Convolutional_Neural_Networks/dataset/training_set'

### Training the model

In [ ]:
classifier.fit_generator(training_set, 
                         steps_per_epoch=250, 
                         validation_data=test_set, 
                         use_multiprocessing=True, 
                         verbose=1,
                         epochs=25, 
                         validation_steps=2000)

Using **Convolutional Neural Networks**, I was able to train a model to detect between cats and dogs.